<a href="https://colab.research.google.com/github/YanaiA/deep-yanai/blob/master/gpt/karpathy_gpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

train_ds = tfds.as_dataframe(tfds.load(name='tiny_shakespeare', split='train'))
val_ds = tfds.as_dataframe(tfds.load(name='tiny_shakespeare', split='validation'))


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/1 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/tiny_shakespeare/1.0.0.incomplete3AYNP7/tiny_shakespeare-train.tfrecord*..…

Generating validation examples...:   0%|          | 0/1 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/tiny_shakespeare/1.0.0.incomplete3AYNP7/tiny_shakespeare-validation.tfreco…

Generating test examples...:   0%|          | 0/1 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/tiny_shakespeare/1.0.0.incomplete3AYNP7/tiny_shakespeare-test.tfrecord*...…

Dataset tiny_shakespeare downloaded and prepared to /root/tensorflow_datasets/tiny_shakespeare/1.0.0. Subsequent calls will reuse this data.


In [ ]:
# lengths = [len(dataset[x]) for x in ['train', 'validation', 'test']]
# print(f"lengths of dataset is {lengths}")

train_txt = train_ds['text'].to_list()[0].decode("utf-8")
print(train_txt[:1000])
val_txt = val_ds['text'].to_list()[0].decode("utf-8")


First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [ ]:
chars = sorted(list(set(train_txt)))
vocab_size = len(chars)
print(''.join(chars))


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


In [ ]:
stoi = {s:i for i, s in enumerate(chars)}
itos = {i:s for i, s in enumerate(chars)}
encode = lambda str:  [stoi[ch] for ch in str]
decode = lambda arr: ''.join([itos[i] for i in arr])

# encoders alternatives: tiktoken (chat-gpt), SentencePiece (google)

print(encode("hello world!"))
print(decode(encode("hello world!")))

[46, 43, 50, 50, 53, 1, 61, 53, 56, 50, 42, 2]
hello world!


In [ ]:
import torch
train_data = torch.tensor(encode(train_txt), dtype=torch.long)
print(train_data[:1000])
val_data = torch.tensor(encode(val_txt), dtype=torch.long)

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
        47, 59, 57,  1, 47, 57,  1, 41, 

In [ ]:
block_size = 8
x = train_data[:block_size*2]
y = train_data[1:block_size*2+1]
for t in range(block_size*2):
    context = x[max(0,t-block_size+1):t+1]
    target = y[t]
    print(f'when context is {context}, target is {target}')

when context is tensor([18]), target is 47
when context is tensor([18, 47]), target is 56
when context is tensor([18, 47, 56]), target is 57
when context is tensor([18, 47, 56, 57]), target is 58
when context is tensor([18, 47, 56, 57, 58]), target is 1
when context is tensor([18, 47, 56, 57, 58,  1]), target is 15
when context is tensor([18, 47, 56, 57, 58,  1, 15]), target is 47
when context is tensor([18, 47, 56, 57, 58,  1, 15, 47]), target is 58
when context is tensor([47, 56, 57, 58,  1, 15, 47, 58]), target is 47
when context is tensor([56, 57, 58,  1, 15, 47, 58, 47]), target is 64
when context is tensor([57, 58,  1, 15, 47, 58, 47, 64]), target is 43
when context is tensor([58,  1, 15, 47, 58, 47, 64, 43]), target is 52
when context is tensor([ 1, 15, 47, 58, 47, 64, 43, 52]), target is 10
when context is tensor([15, 47, 58, 47, 64, 43, 52, 10]), target is 0
when context is tensor([47, 58, 47, 64, 43, 52, 10,  0]), target is 14
when context is tensor([58, 47, 64, 43, 52, 10,  

In [ ]:
batch_size = 4
torch.manual_seed(0)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

def get_batch(split):
    data = train_data if split == 'train' else val_data
    batch_indexes = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in batch_indexes])
    y = torch.stack([data[i+1:i+block_size+1] for i in batch_indexes])
    x = x.to(device)
    y = y.to(device)
    return x, y

x_batch, y_batch = get_batch('train')
print(x_batch)
print(y_batch)

tensor([[46, 43,  1, 45, 47, 60, 43, 57],
        [51,  1, 58, 46, 43,  1, 63, 53],
        [39, 50, 50,  1, 63, 53, 59,  1],
        [53, 61, 52,  1, 57, 53, 59, 50]])
tensor([[43,  1, 45, 47, 60, 43, 57,  1],
        [ 1, 58, 46, 43,  1, 63, 53, 59],
        [50, 50,  1, 63, 53, 59,  1, 46],
        [61, 52,  1, 57, 53, 59, 50,  1]])


In [ ]:
# eval_iters = 200

# @torch.no_grad()  # tells pytorch that no back-prop is done here, so it can be more efficient
# def estimate_loss():
#     out = {}
#     model.eval()  # set to evaluate mode (e.g. no batch-norm, dropout)
#     for split in ['train', 'val']:
#         losses = torch.zeros(eval_iters)
#         for k in range(eval_iters):
#             xb, yb = get_batch(split)
#             _, loss = model(xb, yb)
#             losses[k] = loss.item()
#         out[split] = losses.mean()
    
#     model.train() # return to train mode
#     return out

In [ ]:
import torch.nn as nn
from torch.nn import functional as F
n_embedding = 6 * 16 * 4    # =384
num_heads = 6
block_size = 256
factor = 6      # from the transformers paper


class Head(nn.Module):
    """ One head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embedding, head_size, bias=False)
        self.query = nn.Linear(n_embedding, head_size, bias=False)
        self.value = nn.Linear(n_embedding, head_size, bias=False)
        self.register_buffer('triangle_left', torch.tril(torch.ones(block_size, block_size)))
    
    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.key(x)
        v = self.value(x)

        weights = q @ k.transpose(-2, -1) * (C**-0.5)
        weights = weights.masked_fill(self.triangle_left[:T, :T] == 0, float('-inf'))
        weights = F.softmax(weights, dim=-1)

        out = weights @ v
        return out


class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embedding, n_embedding)

    def forward(self, x):
        concatenated_heads = torch.cat([h(x) for h in self.heads], dim=-1)
        projected_heads = self.proj(concatenated_heads)
        return projected_heads


class FeadForward(nn.Module):
    def __init__(self, n_embedding):
        super().__init__()
        self.ff = nn.Sequential(
            nn.Linear(n_embedding, n_embedding * factor),
            nn.ReLU(),
            nn.Linear(n_embedding * factor, n_embedding),  # projection
        )

    def forward(self, x):
        return self.ff(x)


class Block(nn.Module):
    def __init__(self, n_embeddings, num_heads):
        super().__init__()
        head_size = n_embeddings // num_heads
        self.soft_attention_heads = MultiHeadAttention(factor, n_embedding // factor)
        self.ffwd = FeadForward(n_embedding)
        self.layer_norm1 = nn.LayerNorm(n_embedding)
        self.layer_norm2 = nn.LayerNorm(n_embedding)
        
    def forward(self, x):
        post_attention = x + self.soft_attention_heads(self.layer_norm1(x))
        post_feed_forward = post_attention + self.ffwd(self.layer_norm2(post_attention))
        return post_feed_forward


class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size) -> None:
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embedding)
        self.position_embedding_table = nn.Embedding(vocab_size, n_embedding)
        self.lang_model_head = nn.Linear(n_embedding, vocab_size)
        #self.soft_attention_head = Head(n_embedding)
        #self.soft_attention_heads = MultiHeadAttention(4, n_embedding // 4)
        #self.ffwd = FeadForward(n_embedding)
        self.blocks = nn.Sequential(
            Block(n_embedding, num_heads),
            Block(n_embedding, num_heads),
            Block(n_embedding, num_heads),
            Block(n_embedding, num_heads),
            nn.LayerNorm(n_embedding),
        )
    
    def forward(self, inputs, targets=None):
        B, T = inputs.shape

        # inputs & targets are [B=batch,T=time=block_size]
        tokens_embedding = self.token_embedding_table(inputs)   # (B, T, C=channels=embedding)
        pos_embedding = self.position_embedding_table(torch.arange(T, device=device))   # integers, [0..T-1]
        tok_and_pos = tokens_embedding + pos_embedding
        #post_attention = self.soft_attention_heads(tok_and_pos)
        #post_feed_forward = self.ffwd(post_attention)
        post_blocks = self.blocks(tok_and_pos)
        logits = self.lang_model_head(post_blocks)   # (B, T, C=channels=vocab_size)

        #reshape for cross-entrooy
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits_reshaped = logits.view(B*T, C)
            targets_reshaped = targets.view(B*T)
            loss = F.cross_entropy(logits_reshaped, targets_reshaped)

        return logits, loss

    def generate(self, inputs, new_tokens):
        outputs = inputs
        for _ in range(new_tokens):
            inputs_cropped = inputs[:, -block_size:]  # so we don't get too far in the history
            logits, _ = self(inputs_cropped)
            logits = logits[:, -1, :]   # take logits of last time step => [B, C]
            probs = F.softmax(logits, dim=-1)
            predictions = torch.multinomial(probs, num_samples=1)  # sample from probabilities => [B, 1]
            outputs = torch.cat((outputs, predictions), dim=1)  # [B, T+1]
        return outputs

        
model = BigramLanguageModel(vocab_size).to(device)
logits, loss = model(x_batch, y_batch)
print(loss)

simple_inputs = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_string = model.generate(simple_inputs, 100)[0]  # 0 - take first (and only) batch
print(decode(generated_string.tolist()))



tensor(4.4438, grad_fn=<NllLossBackward0>)

;pRJtw?q'N;Cm&v$czc$BmrXViuA,JEJm'F$HUAu ykQnMmPUX?ULmu,HS!?mJmO;-$RqhmzZLt,Mk;lsAmqQJ'uWLdiuMEPtmwD


In [ ]:
model = BigramLanguageModel(vocab_size).to(device)

eval_iters = 200


def get_batch(split):
    data = train_data if split == 'train' else val_data
    batch_indexes = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in batch_indexes])
    y = torch.stack([data[i+1:i+block_size+1] for i in batch_indexes])
    x = x.to(device)
    y = y.to(device)
    return x, y

    
@torch.no_grad()  # tells pytorch that no back-prop is done here, so it can be more efficient
def estimate_loss():
    out = {}
    model.eval()  # set to evaluate mode (e.g. no batch-norm, dropout)
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            xb, yb = get_batch(split)
            _, loss = model(xb, yb)
            losses[k] = loss.item()
        out[split] = losses.mean()
    
    model.train() # return to train mode
    return out


learning_rate = 1e-4
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
#losses = []
max_iters = 10000
eval_interval = 500
for step in range(max_iters):
    if step % eval_interval == 0:
        losses = estimate_loss()
        print(f'Step: {step}, Loss: {losses}')

    xb, yb = get_batch('train')
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

    #losses.append(loss.item())

#from matplotlib import pyplot
#pyplot.plot(losses)

generated_string = model.generate(simple_inputs, 1000)[0]  # 0 - take first (and only) batch
print(decode(generated_string.tolist()))

IndexError: ignored

In [ ]:
### Side topic - mathematical trick
B, T, C = 4, 8, 2 # batch, time, channels
x = torch.randint(0,10,(B, T, C)).float()

# we want x(b,t) = mean{i<=t}(x(b,i))

# version 1
xbow = torch.zeros((B, T, C))  # bag of words
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1]
        xbow[b,t] = torch.mean(xprev, 0)

print(x[0])
print(xbow[0])

# version 2
lower_triangle = torch.tril(torch.ones(T, T))
#print(lower_triangle)
lower_triangle_mean = lower_triangle / torch.sum(lower_triangle, dim=1, keepdim=True)
#print(lower_triangle_mean)
xbow2 = torch.zeros((B, T, C))  # bag of words
for b in range(B):
    xbow2[b,:] = lower_triangle_mean @ x[b,:]  # matrix multiplication

print(xbow2[0])

# version 2.1
xbow3 = lower_triangle_mean @ x
print(xbow3[0])

# version 3
lower_triangle_zeros_inf = torch.zeros((T,T)).masked_fill(torch.tril(torch.ones(T, T)) == 0, float('-inf'))
weights = F.softmax(lower_triangle_zeros_inf, dim=1)
print(lower_triangle_zeros_inf, weights)

tensor([[0., 3.],
        [1., 4.],
        [7., 2.],
        [7., 0.],
        [3., 1.],
        [9., 2.],
        [1., 5.],
        [5., 6.]])
tensor([[0.0000, 3.0000],
        [0.5000, 3.5000],
        [2.6667, 3.0000],
        [3.7500, 2.2500],
        [3.6000, 2.0000],
        [4.5000, 2.0000],
        [4.0000, 2.4286],
        [4.1250, 2.8750]])
tensor([[0.0000, 3.0000],
        [0.5000, 3.5000],
        [2.6667, 3.0000],
        [3.7500, 2.2500],
        [3.6000, 2.0000],
        [4.5000, 2.0000],
        [4.0000, 2.4286],
        [4.1250, 2.8750]])
tensor([[0.0000, 3.0000],
        [0.5000, 3.5000],
        [2.6667, 3.0000],
        [3.7500, 2.2500],
        [3.6000, 2.0000],
        [4.5000, 2.0000],
        [4.0000, 2.4286],
        [4.1250, 2.8750]])
tensor([[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., -inf, -inf, -inf, -inf],
        [0., 0.,

In [ ]:
### Single head soft-attention
B, T, C = 1, 8, 32
x = torch.randn(B, T, C)
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)     # pass x through fully-connected layers to get k & q & v
q = query(x)
v = value(X)
weights = q @ k.transpose(-2, -1)   # (B,T,16) & (B, 16, T) => (B, T, T)
triangle_left = torch.tril(torch.ones(T, T))
weights = weights.masked_fill(triangle_left==0, float('-inf'))   # only for the decoder. So information from the future won't get in
weights = F.softmax(weights, dim=-1)
out = weights @ v

print(weights)


tensor([[[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.9875, 0.0125, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.3422, 0.0788, 0.5790, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.3510, 0.3178, 0.0982, 0.2331, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.3175, 0.5526, 0.0333, 0.0240, 0.0726, 0.0000, 0.0000, 0.0000],
         [0.0964, 0.2016, 0.0907, 0.0636, 0.3644, 0.1833, 0.0000, 0.0000],
         [0.0109, 0.0114, 0.2499, 0.0949, 0.2438, 0.0862, 0.3030, 0.0000],
         [0.2661, 0.1815, 0.0905, 0.0643, 0.1406, 0.1190, 0.0578, 0.0802]]],
       grad_fn=<SoftmaxBackward0>)
